In [ ]:
!pip -q install -U "gradio>=4.44,<5" "huggingface_hub<0.26.0"
import os, time; print("Restarting…"); time.sleep(1); os._exit(0)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 33.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 75.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
google-adk 1.14.1 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.
google-cola

In [2]:
!pip -q install "pycaret[full]>=3.0,<4" "gradio>=4.0,<5"

import pycaret, sys, sklearn
print("PyCaret:", pycaret.__version__)
print("sklearn:", sklearn.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 47.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 91.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 79.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━

In [8]:
import pandas as pd
import gradio as gr
from pycaret.classification import load_model, predict_model

# if already loaded, reuse your existing model object `clf`
# otherwise uncomment and point to your model
# clf = load_model("/kaggle/input/pycaret-models-prachi/binclf_final")

feature_cols = [
    "age","job","marital","education","default","housing","loan",
    "contact","month","day_of_week","duration","campaign","pdays","previous",
    "poutcome","emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"
]

inputs = []
num_cols = {"age","duration","campaign","pdays","previous",
            "emp.var.rate","cons.price.idx","cons.conf.idx","euribor3m","nr.employed"}

for c in feature_cols:
    if c in num_cols:
        inputs.append(gr.Number(label=c))
    else:
        inputs.append(gr.Textbox(label=c))

def predict_row(*vals):
    row = pd.DataFrame([dict(zip(feature_cols, vals))])

    # ensure numeric fields are numeric (Gradio sometimes passes None/str)
    for c in num_cols:
        row[c] = pd.to_numeric(row[c], errors="coerce")

    out = predict_model(clf, data=row)

    # be tolerant to different column names across PyCaret versions
    label_col = next((c for c in ["prediction_label", "Label", "label"] if c in out.columns), None)
    score_col = next((c for c in ["prediction_score", "Score", "probability"] if c in out.columns), None)

    label = out[label_col].iloc[0] if label_col else "N/A"
    if score_col:
        try:
            score = float(out[score_col].iloc[0])
            return f"Prediction: {label}  |  Score: {score:.3f}"
        except Exception:
            return f"Prediction: {label}"
    else:
        return f"Prediction: {label}"

demo = gr.Interface(fn=predict_row, inputs=inputs, outputs=gr.Textbox(label="output"),
                    title="Bank Marketing — Binary Classifier")
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://b150ad99025756d55c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
